In [1]:
%matplotlib inline

from IPython.display import display as d
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(rc={'figure.figsize':(12,12)})
sns.set_style("white")
import glob
import json
import os
import sys
import json
import shutil
import os
import sys
import multiprocessing
from pprint import pprint
import shutil
pool = multiprocessing.Pool(multiprocessing.cpu_count())

from sklearn.metrics import *
from sklearn.metrics import plot_confusion_matrix

def calc_metrics(y_true, y_pred):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
        "mcc": matthews_corrcoef(y_true, y_pred),
        "confusion_matrix": confusion_matrix(y_true, y_pred)
    }


In [2]:
def read_json(x):
    with open(x) as xfp:
        return json.load(xfp)



In [6]:
import itertools

def analysis(dataset):

    dr = pd.DataFrame(read_json(f"data/avt_image_db/prediction_{dataset}.json"))

    dr["image"] = dr["image"].apply(lambda x: os.path.basename(os.path.splitext(x)[0]))
    dr["pred"] = dr["prediction"].apply(lambda x: 1 if x > 0.5 else 0)
    d(dr.head())

    dm = dr.copy()
    for x in sorted(list(glob.glob(f"data/avt_image_db/annotations/*_{dataset}.ldjson"))):
        print(x)
        v = []
        with open(x) as xfp:
            for l in xfp:
                v.append(json.loads(l))
        rater = os.path.basename(x).split("_")[0]
        dd = pd.DataFrame(v)
        dd["image"] = dd["image"].apply(lambda x: os.path.basename(os.path.splitext(x)[0]))
        dd[rater] = dd["decision"].apply(lambda x: 1 if x=="yes" else 0)
        dm = pd.merge(dm, dd[["image", rater]], left_on="image", right_on="image")
    d(dm.head())
    
    raters = list(dm.columns.difference(["image", "prediction", "pred"]))
    
    dm["mean"] = (sum([dm[x] for x in raters]) / len(raters)).apply(lambda x: 1 if x > 0.5 else 0)
    d(dm.head())
    dm.to_csv(f"{dataset}_annotations.csv", index=False)
    raters +=["mean"]
    
    _metrics = []
    for x1, x2 in itertools.combinations(raters, 2):
        m = calc_metrics(dm[x1], dm[x2])
        m["comp"] = x1 + "~" + x2
        _metrics.append(m)
    for rater in raters:
        m = calc_metrics(dm["pred"], dm[rater])
        m["comp"] = "pred~" + rater
        _metrics.append(m)
    dmetrics = pd.DataFrame(_metrics)
    d(dmetrics)
    dmetrics[["comp", "accuracy", "precision", "recall", "f1", "mcc"]].round(3).to_latex(f"avt_image_db_{dataset}.tex", index=False)


In [7]:
analysis("rule_of_thirds")

,image,prediction,pred
0,7rufkxj9,3.434896e-06,0
1,crvBFbVE,3.850217e-07,0
2,kOUofVfI,3.752668e-09,0
3,Q9ZTaXS0,9.228650e-06,0
4,1782,2.680042e-11,0


data/avt_image_db/annotations/E1_rule_of_thirds.ldjson
data/avt_image_db/annotations/E2_rule_of_thirds.ldjson
data/avt_image_db/annotations/E3_rule_of_thirds.ldjson
data/avt_image_db/annotations/E4_rule_of_thirds.ldjson
data/avt_image_db/annotations/E5_rule_of_thirds.ldjson


,image,prediction,pred,E1,E2,E3,E4,E5
0,7rufkxj9,3.434896e-06,0,0,0,0,1,0
1,crvBFbVE,3.850217e-07,0,0,0,0,0,0
2,kOUofVfI,3.752668e-09,0,0,0,0,0,0
3,Q9ZTaXS0,9.228650e-06,0,0,0,0,1,0
4,1782,2.680042e-11,0,0,0,1,0,0


,image,prediction,pred,E1,E2,E3,E4,E5,mean
0,7rufkxj9,3.434896e-06,0,0,0,0,1,0,0
1,crvBFbVE,3.850217e-07,0,0,0,0,0,0,0
2,kOUofVfI,3.752668e-09,0,0,0,0,0,0,0
3,Q9ZTaXS0,9.228650e-06,0,0,0,0,1,0,0
4,1782,2.680042e-11,0,0,0,1,0,0,0


,accuracy,precision,recall,f1,mcc,confusion_matrix,comp
0,0.704200,0.708000,0.406897,0.516788,0.351287,"[[611, 73], [258, 177]]",E1~E2
1,0.642538,0.581395,0.287356,0.384615,0.192741,"[[594, 90], [310, 125]]",E1~E3
2,0.618409,0.509662,0.485057,0.497055,0.190094,"[[481, 203], [224, 211]]",E1~E4
3,0.672922,0.771654,0.225287,0.348754,0.281063,"[[655, 29], [337, 98]]",E1~E5
4,0.766756,0.922330,0.436782,0.592824,0.519952,"[[668, 16], [245, 190]]",E1~mean
5,0.716711,0.344186,0.296000,0.318280,0.141401,"[[728, 141], [176, 74]]",E2~E3
6,0.663986,0.347826,0.576000,0.433735,0.228886,"[[599, 270], [106, 144]]",E2~E4
7,0.804290,0.622047,0.316000,0.419098,0.342429,"[[821, 48], [171, 79]]",E2~E5
8,0.871314,0.757282,0.624000,0.684211,0.608811,"[[819, 50], [94, 156]]",E2~mean
9,0.645219,0.280193,0.539535,0.368839,0.171277,"[[606, 298], [99, 116]]",E3~E4


In [8]:
analysis("simplicity")

,image,prediction,pred
0,7rufkxj9,3.830996e-02,0
1,crvBFbVE,1.875058e-02,0
2,kOUofVfI,1.000000e+00,1
3,Q9ZTaXS0,7.786412e-01,1
4,1782,4.097230e-07,0


data/avt_image_db/annotations/E1_simplicity.ldjson
data/avt_image_db/annotations/E2_simplicity.ldjson
data/avt_image_db/annotations/E3_simplicity.ldjson
data/avt_image_db/annotations/E4_simplicity.ldjson
data/avt_image_db/annotations/E5_simplicity.ldjson


,image,prediction,pred,E1,E2,E3,E4,E5
0,7rufkxj9,3.830996e-02,0,1,1,1,1,1
1,crvBFbVE,1.875058e-02,0,0,1,0,1,0
2,kOUofVfI,1.000000e+00,1,1,1,0,1,1
3,Q9ZTaXS0,7.786412e-01,1,0,1,0,0,0
4,1782,4.097230e-07,0,1,1,1,1,1


,image,prediction,pred,E1,E2,E3,E4,E5,mean
0,7rufkxj9,3.830996e-02,0,1,1,1,1,1,1
1,crvBFbVE,1.875058e-02,0,0,1,0,1,0,0
2,kOUofVfI,1.000000e+00,1,1,1,0,1,1,1
3,Q9ZTaXS0,7.786412e-01,1,0,1,0,0,0,0
4,1782,4.097230e-07,0,1,1,1,1,1,1


,accuracy,precision,recall,f1,mcc,confusion_matrix,comp
0,0.689008,0.525758,0.908377,0.666027,0.466281,"[[424, 313], [35, 347]]",E1~E2
1,0.808758,0.836000,0.547120,0.661392,0.559500,"[[696, 41], [173, 209]]",E1~E3
2,0.657730,0.499255,0.876963,0.636277,0.407484,"[[401, 336], [47, 335]]",E1~E4
3,0.821269,0.791667,0.646597,0.711816,0.590470,"[[672, 65], [135, 247]]",E1~E5
4,0.915996,0.852941,0.910995,0.881013,0.817261,"[[677, 60], [34, 348]]",E1~mean
5,0.610366,0.948000,0.359091,0.520879,0.390591,"[[446, 13], [423, 237]]",E2~E3
6,0.781055,0.809240,0.822727,0.815928,0.545966,"[[331, 128], [117, 543]]",E2~E4
7,0.638963,0.910256,0.430303,0.584362,0.405084,"[[431, 28], [376, 284]]",E2~E5
8,0.751564,0.968137,0.598485,0.739700,0.582656,"[[446, 13], [265, 395]]",E2~mean
9,0.573727,0.330849,0.888000,0.482085,0.315661,"[[420, 449], [28, 222]]",E3~E4
